# K-Truss
# Does not run on CUDA 11.4


In this notebook, we will use cuGraph to identify the K-Truss clusters in a test graph  

Notebook Credits
* Original Authors: Bradley Rees
* Created:   10/28/2019
* Last Edit: 06/22/2022


## Introduction

Compute the k-truss of the graph G.  A K-Truss is a relaxed cliques where every vertex is supported by at least k-2 triangle.
NOTE: k-truss is currently not supported on CUDA 11.4 systems.

Ref:

[1] Cohen, J.,
    "Trusses: Cohesive subgraphs for social network analysis"
    National security agency technical report, 2008

[2] O. Green, J. Fox, E. Kim, F. Busato, et al.
    “Quickly Finding a Truss in a Haystack”
    IEEE High Performance Extreme Computing Conference (HPEC), 2017
    https://doi.org/10.1109/HPEC.2017.8091038

[3] O. Green, P. Yalamanchili, L.M. Munguia,
    “Fast Triangle Counting on GPU”
    Irregular Applications: Architectures and Algorithms (IA3), 2014
    

To compute the K-Truss cluster in cuGraph use: <br>
* __gc = cugraph.ktruss_subgraph(G,k=None, use_weights=True)__
    G : cuGraph.Graph
        cuGraph graph descriptor with connectivity information. k-Trusses are
        defined for only undirected graphs as they are defined for
        undirected triangle in a graph.

    k : int
        The desired k to be used for extracting the k-truss subgraph.

    use_weights : Bool
        whether the output should contain the edge weights if G has them
    
Returns:
    G_truss : cuGraph.Graph
        A cugraph graph descriptor with the k-truss subgraph for the given k.


### Some notes about vertex IDs...
* The current version of cuGraph requires that vertex IDs be representable as 32-bit integers, meaning graphs currently can contain at most 2^32 unique vertex IDs. However, this limitation is being actively addressed and a version of cuGraph that accommodates more than 2^32 vertices will be available in the near future.
* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


### Prep

In [21]:
# Import needed libraries
import cugraph
import cudf

# Import a built-in dataset
from cugraph.experimental.datasets import karate

### Create a Graph

In [25]:
# Create a graph using the imported Dataset object
G = karate.get_graph()
G = G.to_undirected()

In [26]:
print("Main Graph")
print("\tNumber of Vertices: " + str(G.number_of_vertices()))
print("\tNumber of Edges:    " + str(G.number_of_edges()))

Main Graph
	Number of Vertices: 34
	Number of Edges:    78


### Now run K-Truss

In [27]:
# Call k-cores on the graph
kcg = cugraph.ktruss_subgraph(G, 3) 

In [28]:
print("K-Truss Graph")
print("\tNumber of Vertices: " + str(kcg.number_of_vertices()))
print("\tNumber of Edges:    " + str(kcg.number_of_edges()))

K-Truss Graph
	Number of Vertices: 32
	Number of Edges:    67


### Let's looks at the results
The results show that the roughly 2/3s of the edges have been removed.  
Let's look at the degrees of the vertices

In [29]:
d = kcg.degrees()
d.sort_values(by='out_degree', ascending=False)

,in_degree,out_degree,vertex
0,14,14,1
1,14,14,34
2,12,12,33
3,8,8,2
4,7,7,3
5,6,6,4
6,5,5,9
7,5,5,32
8,4,4,6
9,4,4,7


In [30]:
# We can also just get a list of all the remaining edges as COO
coo = kcg.view_edge_list()

In [31]:
# print out edge list
coo

,src,dst
0,5,11
1,21,33
2,2,3
3,7,17
4,32,34
...,...,...
62,3,33
63,29,32
64,25,32
65,4,13


### Just for fun
Let's try specifying a larger K value.  

In [32]:
# Call k-cores on the graph
kcg2 = cugraph.ktruss_subgraph(G, k=5) 

In [33]:
print("K-Truss Graph")
print("\tNumber of Vertices: " + str(kcg2.number_of_vertices()))
print("\tNumber of Edges:    " + str(kcg2.number_of_edges()))

K-Truss Graph
	Number of Vertices: 6
	Number of Edges:    14


___
Copyright (c) 2019-2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___